In [7]:
import pandas as pd

In [8]:
test_unseen_group = pd.read_csv('../data/test_unseen.csv')
train_group = pd.read_csv('../data/train_group.csv').fillna('')
val_seen_group = pd.read_csv('../data/val_seen_group.csv').fillna('')
val_unseen_group = pd.read_csv('../data/val_unseen_group.csv').fillna('')
subgroups = pd.read_csv('../data/subgroups.csv').fillna('')
users = pd.read_csv('../data/users.csv').fillna('')

In [9]:
popular_subgroup = {}
for sub in train_group.subgroup:
    for s in sub.split(' '):
        if(s == ''):
            continue
        try:
            popular_subgroup[s] += 1
        except:
            popular_subgroup[s] = 1
for sub in val_seen_group.subgroup:
    for s in sub.split(' '):
        if(s == ''):
            continue
        try:
            popular_subgroup[s] += 128
        except:
            popular_subgroup[s] = 128
for sub in val_unseen_group.subgroup:
    for s in sub.split(' '):
        if(s == ''):
            continue
        try:
            popular_subgroup[s] += 128
        except:
            popular_subgroup[s] = 128
popular_subgroup_list = sorted(list(popular_subgroup.items()), key=lambda x:x[1], reverse=True)
popular_subgroup_index = [x[0] for x in popular_subgroup_list]

In [10]:
subgroups = subgroups.set_index('subgroup_name').to_dict()['subgroup_id']

In [11]:
user_interest_group = {}
for user,interest in zip(users.user_id, users.interests):
    temp = []
    if(interest == ''):
        user_interest_group[user] = []
        continue
    for i in interest.split(','):
        _, x = i.split('_')
        try:
            temp.append(str(subgroups[x]))
        except:
            continue
    user_interest_group[user] = temp

In [12]:
with open('./submit_unseen.csv', 'w') as f:
    f.write('user_id,subgroup\n')
    for user in test_unseen_group.user_id:
        temp = sorted(user_interest_group[user], key=popular_subgroup_index.index)
        temp = temp[:min(12, len(temp))]
        for i in popular_subgroup_index:
            if(i in temp):
                continue
            temp.append(i)
        rec = temp[:50]
        f.write('{},{}\n'.format(user, ' '.join(rec)))